In [0]:
%python
from pyspark.sql.functions import split, size,max

df = spark.read.format("csv") \
    .option("inferSchema", "true") \
    .option('compression', 'gzip') \
    .load('s3://consumer-orbit-us-east-1-holding/October2024_EMAIL/*.txt.gz')  


df_split = df.withColumn("split_col", split(df["_c0"], "\t"))

df_split_size=df_split.select('split_col',size('split_col').alias("No_of_Columns"))

MaxNumElements=df_split_size.agg({'No_of_Columns': 'max'}).collect()[0][0]


def arraySplitIntoCols (df,No_of_Columns):
    for i in range(No_of_Columns):
        df = df.withColumn(f"col{i}", df.split_col[i])
    
    return df    

df_final=arraySplitIntoCols(df_split_size,MaxNumElements)

df_final.drop('split_col').drop('No_of_Columns').display() 

In [0]:

df = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('s3://consumer-orbit-us-east-1-holding/Total_Consumer_Insights_Email_Counts (4).xlsx')

display(df)





In [0]:
from pyspark.sql.functions import col, StringType

columns= df.columns
display(spark.createDataFrame(columns, StringType()).toDF('column_name'))

In [0]:
df=df.select('_c1')

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

# Add an index column to the DataFrame
df_with_index = df.withColumn("index", monotonically_increasing_id())

# Find the index of the row containing "Field Name"
field_name_index = df_with_index.filter(df_with_index['_c1'] == 'Field Name').select('index').collect()[0][0]

# Filter the DataFrame to include only rows after the "Field Name" row
df_filtered = df_with_index.filter(df_with_index['index'] > field_name_index).drop('index')

df_filtered.dropna().display()  

In [0]:
df_with_index.display()